In [ ]:
#Defining the model

model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)


#Training settings

optimizer = optim.SGD(model.parameters(),lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.1)



checkpoint_dir = './checkpoint/'+args.tag+'/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)



print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

#Training
metrics = {'train_loss':[],  'val_loss':[]}
constant_metrics = {'min_val_epoch':-1, 'min_val_loss':9999999999999999}

def train(epoch):
    global metrics,loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1


    for cnt,batch in enumerate(loader_train):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        optimizer.zero_grad()
        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)



        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip_grad)


            optimizer.step()
            #Metrics
            loss_batch += loss.item()
            print('TRAIN:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['train_loss'].append(loss_batch/batch_count)




def vald(epoch):
    global metrics,loader_val,constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1

    for cnt,batch in enumerate(loader_val):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            #Metrics
            loss_batch += loss.item()
            print('VALD:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['val_loss'].append(loss_batch/batch_count)

    if  metrics['val_loss'][-1]< constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] =  metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(),checkpoint_dir+'val_best.pth')  # OK
        check_test_performance()

print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()


    print('*'*30)
    print('Epoch:',args.tag,":", epoch)
    for k,v in metrics.items():
        if len(v)>0:
            print(k,v[-1])


    print(constant_metrics)
    print('*'*30)

    with open(checkpoint_dir+'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir+'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)



Data and model loaded
Checkpoint dir: ./checkpoint/social-tag/
Training started ...
TRAIN: 	 Epoch: 0 	 Loss: 0.017905406653881073
TRAIN: 	 Epoch: 0 	 Loss: 0.01704898662865162
TRAIN: 	 Epoch: 0 	 Loss: 0.01653238416959842
TRAIN: 	 Epoch: 0 	 Loss: 0.0161204282194376
TRAIN: 	 Epoch: 0 	 Loss: 0.01576061323285103
TRAIN: 	 Epoch: 0 	 Loss: 0.015433841695388159
TRAIN: 	 Epoch: 0 	 Loss: 0.015113428235054016
TRAIN: 	 Epoch: 0 	 Loss: 0.014816240523941815
TRAIN: 	 Epoch: 0 	 Loss: 0.014531371908055412
TRAIN: 	 Epoch: 0 	 Loss: 0.014240691997110843
TRAIN: 	 Epoch: 0 	 Loss: 0.013945008882067421
TRAIN: 	 Epoch: 0 	 Loss: 0.013651493548726043
TRAIN: 	 Epoch: 0 	 Loss: 0.01337091288027855
TRAIN: 	 Epoch: 0 	 Loss: 0.01307677710428834
TRAIN: 	 Epoch: 0 	 Loss: 0.012784409647186598
TRAIN: 	 Epoch: 0 	 Loss: 0.012504966114647686
TRAIN: 	 Epoch: 0 	 Loss: 0.012229107758578132
TRAIN: 	 Epoch: 0 	 Loss: 0.011943436150128642
TRAIN: 	 Epoch: 0 	 Loss: 0.011900724969887713
VALD: 	 Epoch: 0 	 Loss: 0.006

100%|██████████| 602/602 [00:32<00:00, 18.70it/s]

Testing ....


ADE: 1.2063701968762728  FDE: 1.6796964878864178
**************************************************
******************************
Epoch: social-tag : 0
train_loss 0.011900724969887713
val_loss 0.011174220487105944
{'min_val_epoch': 0, 'min_val_loss': 0.011174220487105944}
******************************
TRAIN: 	 Epoch: 1 	 Loss: 0.006804140284657478
TRAIN: 	 Epoch: 1 	 Loss: 0.0066112675704061985
TRAIN: 	 Epoch: 1 	 Loss: 0.006436558905988932
TRAIN: 	 Epoch: 1 	 Loss: 0.006359525490552187
TRAIN: 	 Epoch: 1 	 Loss: 0.006419632025063038
TRAIN: 	 Epoch: 1 	 Loss: 0.006731942916909854
TRAIN: 	 Epoch: 1 	 Loss: 0.006728938980294126
TRAIN: 	 Epoch: 1 	 Loss: 0.0065947421244345605
TRAIN: 	 Epoch: 1 	 Loss: 0.006387129063821501
TRAIN: 	 Epoch: 1 	 Loss: 0.006208326434716582
TRAIN: 	 Epoch: 1 	 Loss: 0.00607248078185049
TRAIN: 	 Epoch: 1 	 Loss: 0.005951552108551065
TRAIN: 	 Epoch: 1 	 Loss: 0.00578877363855449
TRAIN: 	 Epoch: 1 	 Loss: 0.00568496181430029
TRAIN: 	 Epoch: 1 	 Loss: 0.0057473114

100%|██████████| 602/602 [00:32<00:00, 18.72it/s]


...

ADE: 0.40840905899315544  FDE: 0.7064753683608003
**************************************************
******************************
Epoch: social-tag : 135
train_loss -0.01137603107682597
val_loss -0.013215723215055859
{'min_val_epoch': 135, 'min_val_loss': -0.013215723215055859}
******************************
TRAIN: 	 Epoch: 136 	 Loss: -0.012927771545946598
TRAIN: 	 Epoch: 136 	 Loss: -0.012659152504056692
TRAIN: 	 Epoch: 136 	 Loss: -0.012495587579905987
TRAIN: 	 Epoch: 136 	 Loss: -0.011971110245212913
TRAIN: 	 Epoch: 136 	 Loss: -0.012093967013061046
TRAIN: 	 Epoch: 136 	 Loss: -0.01196959444011251
TRAIN: 	 Epoch: 136 	 Loss: -0.011822906189731188
TRAIN: 	 Epoch: 136 	 Loss: -0.01179398933891207
TRAIN: 	 Epoch: 136 	 Loss: -0.01166304728637139
TRAIN: 	 Epoch: 136 	 Loss: -0.01152402488514781
TRAIN: 	 Epoch: 136 	 Loss: -0.011486058462072502
TRAIN: 	 Epoch: 136 	 Loss: -0.011573687583828965
TRAIN: 	 Epoch: 136 	 Loss: -0.01158976311293932
TRAIN: 	 Epoch: 136 	 Loss: -0.011524014108

100%|██████████| 602/602 [00:32<00:00, 18.75it/s]

Testing ....


ADE: 0.3818638188755609  FDE: 0.606985072941723
**************************************************
******************************
Epoch: social-tag : 141
train_loss -0.011314775444997168
val_loss -0.013516417022578973
{'min_val_epoch': 141, 'min_val_loss': -0.013516417022578973}
******************************
TRAIN: 	 Epoch: 142 	 Loss: -0.011996381916105747
TRAIN: 	 Epoch: 142 	 Loss: -0.01252239290624857
TRAIN: 	 Epoch: 142 	 Loss: -0.012779214108983675
TRAIN: 	 Epoch: 142 	 Loss: -0.012641215464100242
TRAIN: 	 Epoch: 142 	 Loss: -0.012040808238089085
TRAIN: 	 Epoch: 142 	 Loss: -0.01177009909103314
TRAIN: 	 Epoch: 142 	 Loss: -0.011793331908328193
TRAIN: 	 Epoch: 142 	 Loss: -0.011905087623745203
TRAIN: 	 Epoch: 142 	 Loss: -0.01196788458360566
TRAIN: 	 Epoch: 142 	 Loss: -0.01200602762401104
TRAIN: 	 Epoch: 142 	 Loss: -0.011744501407850872
TRAIN: 	 Epoch: 142 	 Loss: -0.01164160684371988
TRAIN: 	 Epoch: 142 	 Loss: -0.011602583341300488
TRAIN: 	 Epoch: 142 	 Loss: -0.0117070491292

 76%|███████▌  | 456/602 [00:24<00:05, 24.73it/s]